In [11]:
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.options.display.max_columns = None


In [2]:
url = "http://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=%s&RangeType=2&Season=2015-16&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"
#EVENTMSGTYPE = 2 = Missed Shot
#EVENTMSGTYPE = 4 = Rebound
#PERSON1TYPE = 0 = End/Start of Quarter
#PERSON1TYPE = 1 = ? not too important
#PERSON1TYPE = 2 = Home Timeout/violation/misc
#PERSON1TYPE = 3 = Visitor Timeout/violation/misc
#PERSON1TYPE = 4 = Home Player
#PERSON1TYPE = 5 = Visitor Player


In [3]:
gameInts = range(21500001, 21501231)#21501231
playsList = []
headers = []
# get all of the plays for 2015-16 regular season.
# commented out and saved as csv to save time
"""
for i in gameInts:
    currID = "00%d"%i
    currUrl = url % currID 
    #had to add user-agent header so api would accept my request from http://stackoverflow.com/questions/35009755/python-get-raises-httperror-400-client-error-but-after-manually-accessing-url
    response = requests.get(currUrl, headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0'})
    respCode = response.status_code 
    if respCode == 200:
        headers = response.json()['resultSets'][0]['headers']
        plays = response.json()['resultSets'][0]['rowSet']
        for p in plays:
            playsList.append(p)
    

playsDF = pd.DataFrame(playsList, columns=headers)

"""


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,0021500001,0,12,0,1,8:12 PM,12:00,None,None,None,...,None,None,None,0,0,None,NaN,None,None,None
1,0021500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,None,None,...,Detroit,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET
2,0021500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),None,MISS Drummond 2' Driving Layup,...,None,None,None,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL
3,0021500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),None,None,...,None,None,None,0,0,None,NaN,None,None,None
4,0021500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,None,None,...,None,None,None,0,0,None,NaN,None,None,None


In [13]:
playsDF.head(20)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,0021500001,0,12,0,1,8:12 PM,12:00,None,None,None,None,None,0.0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
1,0021500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,None,None,None,None,4.0,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL,5,203083,Andre Drummond,1.610613e+09,Detroit,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET
2,0021500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),None,MISS Drummond 2' Driving Layup,None,None,5.0,203083,Andre Drummond,1.610613e+09,Detroit,Pistons,DET,0,0,None,NaN,None,None,None,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL
3,0021500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),None,None,None,None,4.0,203145,Kent Bazemore,1.610613e+09,Atlanta,Hawks,ATL,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
4,0021500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,None,None,None,None,4.0,203145,Kent Bazemore,1.610613e+09,Atlanta,Hawks,ATL,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
5,0021500001,5,1,80,1,8:13 PM,11:21,None,None,Marc Morris 13' Step Back Jump Shot (2 PTS) (D...,2 - 0,-2,5.0,202694,Marcus Morris,1.610613e+09,Detroit,Pistons,DET,5,203083,Andre Drummond,1.610613e+09,Detroit,Pistons,DET,0,0,None,NaN,None,None,None
6,0021500001,6,1,80,1,8:13 PM,11:00,Millsap 12' Step Back Jump Shot (2 PTS) (Korve...,None,None,2 - 2,TIE,4.0,200794,Paul Millsap,1.610613e+09,Atlanta,Hawks,ATL,4,2594,Kyle Korver,1.610613e+09,Atlanta,Hawks,ATL,0,0,None,NaN,None,None,None
7,0021500001,7,1,101,1,8:14 PM,10:44,None,None,Caldwell-Pope 9' Driving Floating Jump Shot (2...,4 - 2,-2,5.0,203484,Kentavious Caldwell-Pope,1.610613e+09,Detroit,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET,0,0,None,NaN,None,None,None
8,0021500001,8,2,1,1,8:14 PM,10:27,MISS Horford 20' Jump Shot,None,None,None,None,4.0,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
9,0021500001,9,4,0,1,8:14 PM,10:25,None,None,Jackson REBOUND (Off:0 Def:1),None,None,5.0,202704,Reggie Jackson,1.610613e+09,Detroit,Pistons,DET,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None


In [19]:
playsDF.tail()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
568328,0021501230,558,3,11,4,12:51 AM,0:25,None,None,Nurkic Free Throw 1 of 2 (11 PTS),99 - 107,8,5.0,203994,Jusuf Nurkic,1.610613e+09,Denver,Nuggets,DEN,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
568329,0021501230,560,3,12,4,12:52 AM,0:25,None,None,MISS Nurkic Free Throw 2 of 2,None,None,5.0,203994,Jusuf Nurkic,1.610613e+09,Denver,Nuggets,DEN,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
568330,0021501230,562,4,0,4,12:52 AM,0:23,Connaughton REBOUND (Off:3 Def:5),None,None,None,None,4.0,1626192,Pat Connaughton,1.610613e+09,Portland,Trail Blazers,POR,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None
568331,0021501230,563,6,28,4,12:52 AM,0:21,None,None,Mudiay Personal Take Foul (P1.T4) (B.Adams),None,None,5.0,1626144,Emmanuel Mudiay,1.610613e+09,Denver,Nuggets,DEN,4,203148,Brian Roberts,1.610613e+09,Portland,Trail Blazers,POR,1,0,None,NaN,None,None,None
568332,0021501230,565,13,0,4,12:53 AM,0:00,None,None,None,99 - 107,8,0.0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None


In [8]:
playsDF.to_csv("2015-all-plays")

In [76]:
# get plays by game
gamePlays = playsDF[playsDF['GAME_ID'] == "0021500001"]
gamePlays = gamePlays[np.isfinite(gamePlays['PERSON1TYPE'])]

# get ppp
playIndex = range(1,len(gamePlays))
homePoss = 0
visPoss = 0
homeOREB=0
visOREB = 0
for i in playIndex:
    if gamePlays.iloc[i]['EVENTMSGTYPE'] == 4:
        if gamePlays.iloc[i]['PERSON1TYPE'] == 5.0 and gamePlays.iloc[i-1]['PERSON1TYPE'] == 5.0:
            visOREB += 1
        elif gamePlays.iloc[i]['PERSON1TYPE'] == 4.0 and gamePlays.iloc[i-1]['PERSON1TYPE'] == 4.0:
            homeOREB += 1
    
            
print(homeOREB)
print(visOREB)
        

# get ppp on OREB poss

7
23


In [69]:
print(gamePlays['PERSON1TYPE'].unique())




[ 0.  4.  5.  2.  3.  1.]
